## Access MySQL from python

In [1]:
#!/usr/bin/python

import   pandas          as pd
import   MySQLdb
import   mysql.connector
from     sqlalchemy      import create_engine

# Open database connection
db = MySQLdb.connect("localhost","root","","matrimony" )

# prepare a cursor object using cursor() method
cursor = db.cursor()

# execute SQL query using execute() method.
cursor.execute("SELECT VERSION()")

# Fetch a single row using fetchone() method.
db_version = cursor.fetchone()
print("Database version : %s " % db_version)

# disconnect from server
db.close()

Database version : 10.1.31-MariaDB 


### Access data from mysql table -  database_for_matrimony1

In [2]:
import   mysql.connector, re
import   pandas              as  pd
import   numpy               as  np

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database="matrimony"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM database_for_matrimony1")

myresult = mycursor.fetchall()

mycol = ['S_No','Profile_ID','Age','Gender','Marital_Status','Caste',\
          'Sub_Caste','State','City','Country','Height','physical_status',\
          'Family_Status','Family_Type','Family_Values','Groom_s_Age',\
          'Groom_s_Education','Groom_s_Income','Groom_s_Location',\
          'About_Description']     


df      = pd.DataFrame(myresult, columns= mycol)

print(df.head())  
print(list(df))
print(df.info())

   S_No Profile_ID  Age  Gender Marital_Status     Caste    Sub_Caste  \
0     1   M5929711   27  Female  Never married  Mudaliar    Melakarar   
1     2   M5929712   28  Female  Never married  Mudaliar    Melakarar   
2     3   M5929713   28  Female  Never married     Nayar      Nambiar   
3     4   M5929714   27  Female        widower     Nayar      Nambiar   
4     5   M5929715   27  Female        widower    Kallar  Mukkulathor   

        State     City Country Height physical_status       Family_Status  \
0  Tamil Nadu  Chennai   India    5.8          Normal        Middle Class   
1  Tamil Nadu  Chennai   India    5.8          Normal  Upper Middle Class   
2  Tamil Nadu  Chennai   India    5.6          Normal                Rich   
3  Tamil Nadu  Chennai   India    5.9          Normal            Affluent   
4  Tamil Nadu  Chennai   India    5.8          Normal                Rich   

  Family_Type Family_Values Groom_s_Age Groom_s_Education  Groom_s_Income  \
0       Joint      Or

### Create corpus data from the user search results

In [3]:
file1 = open("Mate_Corpus.txt","w") 
L = ["This corpus is used to assist the users to get their potential matches"]  
 
file1.writelines(L) 

In [4]:
n   =  df.shape[0] # number of rows

Pref_Age_Lo = []
Pref_Age_Up = []

for i in range(n):
    if len(df['Groom_s_Age'][i]) == 5:
        lo, up = re.findall('\d+', df['Groom_s_Age'][i])
    else:
        lo     = df['Age'][i] + 1
        up     = df['Age'][i] + 5
        
    Pref_Age_Lo.append(lo)
    Pref_Age_Up.append(up)

Lower  = pd.Series(Pref_Age_Lo)
Upper  = pd.Series(Pref_Age_Up)

df['Partner_Age_LI']   =   Lower.values
df['Partner_Age_UI']   =   Upper.values

df['Partner_Age_LI'] = df['Partner_Age_LI'].astype('int32')
df['Partner_Age_UI'] = df['Partner_Age_UI'].astype('int32')


print(df.head(3).T)


                               0                   1              2
S_No                           1                   2              3
Profile_ID              M5929711            M5929712       M5929713
Age                           27                  28             28
Gender                    Female              Female         Female
Marital_Status     Never married       Never married  Never married
Caste                   Mudaliar            Mudaliar          Nayar
Sub_Caste              Melakarar           Melakarar        Nambiar
State                 Tamil Nadu          Tamil Nadu     Tamil Nadu
City                     Chennai             Chennai        Chennai
Country                    India               India          India
Height                       5.8                 5.8            5.6
physical_status           Normal              Normal         Normal
Family_Status       Middle Class  Upper Middle Class           Rich
Family_Type                Joint             Nuc

In [5]:
male_df        = df.loc[df['Gender'] == 'Male',['Profile_ID','Age','Gender','Marital_Status','Caste','Sub_Caste','Partner_Age_LI','Partner_Age_UI']]
female_df      = df.loc[df['Gender'] == 'Female',['Profile_ID','Age','Gender','Marital_Status','Caste','Sub_Caste','Partner_Age_LI','Partner_Age_UI']]

print(male_df.shape)
print(male_df.head())
print(female_df.shape)
print(female_df.head(13))


(52, 8)
   Profile_ID  Age Gender    Marital_Status    Caste    Sub_Caste  \
12   M5929723   27   Male  Awaiting Divorce   Thevar  Mukkulathor   
13   M5929724   30   Male          Divorced  Maravar  Mukkulathor   
14   M5929725   30   Male          Divorced   Kallar  Mukkulathor   
15   M5929726   24   Male     Never married    Nayar      Nambiar   
16   M5929727   24   Male     Never married  Maravar  Mukkulathor   

    Partner_Age_LI  Partner_Age_UI  
12              25              27  
13              28              29  
14              28              29  
15              22              23  
16              22              23  
(48, 8)
   Profile_ID  Age  Gender Marital_Status     Caste    Sub_Caste  \
0    M5929711   27  Female  Never married  Mudaliar    Melakarar   
1    M5929712   28  Female  Never married  Mudaliar    Melakarar   
2    M5929713   28  Female  Never married     Nayar      Nambiar   
3    M5929714   27  Female        widower     Nayar      Nambiar   
4    M5

In [6]:
df_col = list(female_df)
print(df_col)

['Profile_ID', 'Age', 'Gender', 'Marital_Status', 'Caste', 'Sub_Caste', 'Partner_Age_LI', 'Partner_Age_UI']


In [7]:
n   =  female_df.shape[0] # number of rows
n   =  10
for i in range(n):    
    user_age        = female_df['Age'][i]
    user_profile_ID = female_df['Profile_ID'][i]

    ### Female preference

    pref_age_l      = female_df['Partner_Age_LI'][i]                                
    pref_age_u      = female_df['Partner_Age_UI'][i]

    msg = "\nRecord: %d Profile_ID %s User Age: %d preferred partner age: Between %d and %d" \
           % (i, user_profile_ID, user_age, int(pref_age_l), int(pref_age_u))
    
    print(msg)
    
    L1     = msg
    file1.writelines(L1) 
    
    print(male_df['Partner_Age_LI'].dtype)
    pref_male_df   = male_df.loc[(male_df['Age'] >=  int(pref_age_l)) & (male_df['Age'] <=  int(pref_age_u)) &\
                                 (male_df['Partner_Age_LI'] <=  int(user_age)) & (male_df['Partner_Age_UI'] >=  int(user_age))]

    print(pref_male_df.head(3).T)
    msg = ' Count of matches: %d' % (pref_male_df['Profile_ID'].count())
    
    L2     = msg
    file1.writelines(L2) 
    
file1.close() #to change file access modes 


Record: 0 Profile_ID M5929711 User Age: 27 preferred partner age: Between 28 and 30
int32
                         17           18           19
Profile_ID         M5929728     M5929729     M5929730
Age                      29           29           30
Gender                 Male         Male         Male
Marital_Status     Divorced     Divorced     Divorced
Caste                Thevar       Kallar      Maravar
Sub_Caste       Mukkulathor  Mukkulathor  Mukkulathor
Partner_Age_LI           27           27           25
Partner_Age_UI           29           29           29

Record: 1 Profile_ID M5929712 User Age: 28 preferred partner age: Between 29 and 30
int32
                         13           14           17
Profile_ID         M5929724     M5929725     M5929728
Age                      30           30           29
Gender                 Male         Male         Male
Marital_Status     Divorced     Divorced     Divorced
Caste               Maravar       Kallar       Thevar
Sub_Cast

In [8]:
import nltk
import warnings
warnings.filterwarnings("ignore")

# nltk.download() # for downloading packages

import numpy as np
import random
import string # to process standard python strings

In [ ]:
### Load Corpus data from Mate_Corpus.txt

f                    =  open('Mate_Corpus.txt','r',errors = 'ignore')
lines                =  list(f)
List_lines_lower     =  []

for line in lines:
    line_count = len(line)
    
    if  line_count > 71:
        
        line       = line.lower();
        print("\nLine %s" %line)
        List_lines_lower.append(line)

f.close()
print(List_lines_lower)
#nltk.download('punkt') # first-time use only

#nltk.download('wordnet') # first-time use only


Line record: 0 profile_id m5929711 user age: 27 preferred partner age: between 28 and 30 count of matches: 11


Line record: 1 profile_id m5929712 user age: 28 preferred partner age: between 29 and 30 count of matches: 12


Line record: 2 profile_id m5929713 user age: 28 preferred partner age: between 29 and 30 count of matches: 12


Line record: 3 profile_id m5929714 user age: 27 preferred partner age: between 28 and 30 count of matches: 11


Line record: 4 profile_id m5929715 user age: 27 preferred partner age: between 28 and 30 count of matches: 11


Line record: 5 profile_id m5929716 user age: 25 preferred partner age: between 26 and 30 count of matches: 14


Line record: 6 profile_id m5929717 user age: 24 preferred partner age: between 25 and 30 count of matches: 13


Line record: 7 profile_id m5929718 user age: 25 preferred partner age: between 26 and 30 count of matches: 14


Line record: 8 profile_id m5929719 user age: 25 preferred partner age: between 26 and 30 count of match

In [ ]:
raw         = List_lines_lower[0]

sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 

word_tokens = nltk.word_tokenize(raw)# converts to list of words
sent_tokens[:2]
word_tokens[:5]

lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):

    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):

    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)

GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

# Checking for greetings

def greeting(sentence):

    """If user's input is a greeting, return a greeting response"""

    for word in sentence.split():

        if word.lower() in GREETING_INPUTS:

            return random.choice(GREETING_RESPONSES)

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

# Generating response

def response(user_response):

    robo_response=''

    sent_tokens.append(user_response)

    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

    tfidf = TfidfVec.fit_transform(sent_tokens)

    vals = cosine_similarity(tfidf[-1], tfidf)

    idx=vals.argsort()[0][-2]

    flat = vals.flatten()

    flat.sort()

    req_tfidf = flat[-2]

    if(req_tfidf==0):

        robo_response=robo_response+"I am sorry! I don't understand you"

        return robo_response

    else:

        robo_response = robo_response+sent_tokens[idx]

        return robo_response

flag=True

print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")



while(flag==True):

    user_response = input()

    user_response=user_response.lower()

    if(user_response!='bye'):

        if(user_response=='thanks' or user_response=='thank you' ):

            flag=False

            print("ROBO: You are welcome..")

        else:

            if(greeting(user_response)!=None):

                print("ROBO: "+greeting(user_response))

            else:

                print("ROBO: ",end="")

                print(response(user_response))

                sent_tokens.remove(user_response)

    else:

        flag=False

        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
Count
ROBO: record: 0 profile_id m5929711 user age: 27 preferred partner age: between 28 and 30 count of matches: 11
